# World Info Tuning
mkultra provides a simplified trainer for generating soft World Info*.

This trainer takes datapoints in a "call and response" format. The "call" is a fixed prompt (e.g. "A detailed description of the Spider Tank:") that should elicit the "response" (a few paragraphs of information about the Spider Tank). Loss is only calculated against the responses.

The trainer will optimize a soft prompt such that the call is followed by the response. The combined call and response is moved randomly within the context window to prevent overfitting, and the gap is filled with random tokens.

This setup may work for other tasks, but if your needs are significantly different then it is recommended to see tuning_finetune.ipynb for tips on rolling your own.

*Short prompt infixes for AI text adventures and writing.
Describes a character, subject, artistic direction, etc. with the intent of steering the output to consistently make use of the details in question.

In [1]:
#@title Setup for Colab only
#!pip install transformers
#!pip install git+https://github.com/corolla-johnson/mkultra.git#egg=mkultra --log PIP_LOG

In [2]:
from transformers.pipelines import pipeline
from mkultra.models.tuning import GPT2PromptTuningLM
from mkultra.tokenizers import GPT2TokenizerFast
from mkultra.soft_prompt import SoftPrompt
from transformers import Adafactor
from mkultra.trainers import WorldInfoTrainer
import torch
import json

In [3]:
# Use an mkultra prompt tuning LM and a standard tokenizer.
model = GPT2PromptTuningLM.from_pretrained("gpt2").to("cuda")
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [4]:
# Set the length of the soft prompt.
# The paper does not recomment going over 100 tokens.
model.initialize_soft_prompt(n_tokens=20)

In [15]:
sp = SoftPrompt.from_file("sample_sps/world_info/emma_gpt2.json")
model.set_soft_prompt(sp)

In [7]:
with open("datasets/emma.json") as file:
    blocks = json.load(file)

print(blocks)

[{'call': 'Detailed description of Emma Violence:\n', 'response': 'Emma Violence is a British cybernetic assassin who is known for her elegant style and high-profile targets. She is described to be a perfectionist in her work, often going above and beyond the call of duty. She has two guns implanted to her forearms and can utilize them both with deadly accuracy. Despite her cold, ruthless, and calculating nature, she has a warm and motherly side that she only shows to a select few people.'}, {'call': 'Detailed description of Emma Violence:\n', 'response': 'Emma Violence is a British cybernetic assassin who is known for her elegant style and high-profile targets. She is described to be a perfectionist in her work, often going above and beyond the call of duty. She has two guns implanted to her forearms and can utilize them both with deadly accuracy. Despite her cold, ruthless, and calculating nature, she has a warm and motherly side that she only shows to a select few people.'}]


In [8]:
# WorldInfoTrainer will help prevent overfitting by moving the target
# up and down the context, filling the rest with random tokens.
# Also, only the 'response' string is used to calculate loss.
trainer = WorldInfoTrainer(
    model=model,
    tokenizer=tokenizer,
    optimizer=Adafactor(params=[model.get_soft_params()]),
    blocks=blocks,
    max_spacing=100,
    min_loss=0.4
)

In [9]:
trainer.train(1)

0/2: Loss: 0.7642713189125061
1/2: Loss: 0.5425863265991211
C:\Users\STARSTRUCK\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\optimization.py:562: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:1005.)
  exp_avg_sq_row.mul_(beta2t).add_(1.0 - beta2t, update.mean(dim=-1))


In [16]:
call = tokenizer("A detailed dscription of Emma Violence:\n", return_tensors="pt").input_ids.cuda()

model.eval()

basic_output = model.generate(
    call,
    do_sample=False,
    min_length=call.shape[-1] + 100,
    max_length=call.shape[-1] + 100,
    pad_token_id=tokenizer.eos_token_id
)
print(tokenizer.decode(basic_output[0]))

A detailed dscription of Emma Violence:
Emma Violence is a British cybernetic assassin who is known for her elegant style and high-profile targets. She is described to be a perfectionist in her work, often going above and beyond the call of duty. Despite her cold, ruthless, and calculating nature, she has a warm and motherly side that she often goes above and beyond the call of duty. Despite her cold, ruthless, and calculating nature, she has a warm and motherly side that she often goes above and beyond the


In [14]:
metadata = { "name" : "Emma Violence",
             "description" : "A British cybernetic assassin" }

torch.save(model.get_soft_params(), "emma.pt")

sp = SoftPrompt.from_tuning_model(model, metadata)
sp.to_file("sample_sps/world_info/emma_gpt2.json")

## Best Practices for World Info
This is fresh territory and the example here is likely far from optimal.
Here are potential techniques to explore:
- Define bad_words_ids (e.g. square brackets) and surround the response in them to keep the output from repeating the training data verbatim.
- Rather than describing the subject in grammatically correct sentences, try a word cloud approach.
- Rather than describing the subject, provide implicit examples of to write about it ("John doffed his black top hat and cleaned it with his white handkerchief")
- Consider the use of the subject's proper noun vs its pronouns. (Should you start every sentence with "John Doe", or a mix of "John" and "He"?)
- Write your own trainer?